In [1]:
import os, sys, time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

In [3]:
pd.options.display.float_format = '{:0,.1f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

# SOV Travel Time
Identified Corridors
AM Peak Corridors

In [5]:
df = pd.read_csv(r'C:\Workspace\sc_2018_rtp_TEST\soundcast\outputs\validation\corridor_speeds.csv')

# For each Corridor_Descrption, define as AM or pm beak by finding which direction has max value
df = df[df['tod'].isin(['7to8','17to18']) & (df['Corridor_Number'] <= 20)]

# Corridors are grouped so that directions are [1,2], [3,4]
for i in range(1,21):
# for i in range(7,8):
    _df = df[df['Corridor_Number'] == i]
#     print(_df)
    peak_time = _df.sort_values('auto_time').iloc[-1]['tod']
    df.loc[df['Corridor_Number']==i, 'peak'] = peak_time
    
df.rename(columns={'Corridor_Description': 'Corridor', 'auto_time':'Auto Travel Time (min.)'}, inplace=True)
df[(df['peak'] == '7to8') & (df['tod'] == '7to8')][['Corridor','Auto Travel Time (min.)']]

,Corridor,Auto Travel Time (min.)
15,Everett to Seattle - SB,70.5
24,Tacoma to Seattle - NB,76.0
51,Lynnwood to Bellevue - SB,42.3
78,Auburn to Renton - NB,28.1
96,Redmond to Seattle - WB,31.3
114,Redmond to Bellevue - WB,14.4
132,Issaquah to Bellevue - WB,15.3
159,Bellevue to Seattle via 520 - EB,25.0
168,Bellevue to Seattle via 90 - WB,28.9


PM Peak Corridors

In [6]:
df[(df['peak'] == '17to18') & (df['tod'] == '17to18')][['Corridor','Auto Travel Time (min.)']]

,Corridor,Auto Travel Time (min.)
2,Seattle to Everett - NB,68.9
29,Seattle to Tacoma - SB,69.5
38,Bellevue to Lynnwood - NB,40.2
56,Tukwila to Bellevue - NB,31.2
65,Bellevue to Tukwila - SB,32.4
83,Renton to Auburn - SB,27.5
101,Seattle to Redmond - EB,29.3
119,Bellevue to Redmond - EB,13.6
137,Bellevue to Issaquah - EB,14.9
146,Seattle to Bellevue via 520 - WB,21.9


# Transit Travel Time
Identified Corridors
AM & PM Peaks

In [7]:
# Use the corridors defined above; 
# Use some old od corridor work to get total transit travel time skims between centers

In [8]:
od_obs = pd.read_sql_table('observed_od_travel_times', 'sqlite:///../../../inputs/db/soundcast_inputs.db')

max_internal_zone = 3700
zone_list = [532,2286,1552,1125]

run_dict = {'run': r'../../..'}

results_dict = {}

for skim_name in ['7to8','17to18']:
    user_class_dict = {}
    for mode in ['a','r']:
        results = pd.DataFrame()

        for run_dir, current_run_name in run_dict.items():
            skims_dir = run_dir + r'\inputs'

            if run_dir == 'run':
                h5_contents = h5py.File(r'..\..\..\inputs\model\roster' + r'\\' + skim_name + '.h5')
            else:
                h5_contents = h5py.File(skims_dir + r'\\' + skim_name + '.h5')
            for zone in zone_list:
                df = pd.DataFrame()

                for user_class in ['ivtw','twtw','auxw']:
                    df[user_class+mode] = h5_contents['Skims'][user_class+mode][:][zone-1,:max_internal_zone]
#                     df['travel_time'] = df[user_class]/100 # skim data is stored in hundreds

                df['otaz'] = zone
                df['dtaz'] = df.index+1
                df = df[df['dtaz'].isin(zone_list)]

                results = results.append(df)
            
        user_class_dict[mode] = results
    results_dict[skim_name] = user_class_dict
    
df_all = pd.DataFrame()
for skim_name in ['7to8','17to18']:
    for mode in ['r','a']:
        results_dict[skim_name][mode]['travel_time'] = results_dict[skim_name][mode][['ivtw'+mode,'twtw'+mode,'auxw'+mode]].sum(axis=1)
        results_dict[skim_name][mode]['travel_time'] = results_dict[skim_name][mode][['ivtw'+mode,'twtw'+mode,'auxw'+mode]].sum(axis=1)/100.0
        results_dict[skim_name][mode]['travel_time']

**AM (7 to 8)**

In [9]:
skim_name = '7to8'

df_all = results_dict[skim_name]['r']
df_all['mode'] = 'r'
df_rail = results_dict[skim_name]['a']
df_rail['mode'] = 'a'
df = pd.merge(df_all,df_rail,on=['otaz','dtaz'], suffixes=['_all','_rail'])
df
# Take minimum travel time 
df['travel_time_min'] = df[['travel_time_all','travel_time_rail']].min(axis=1)

# # join observed data to get 
od_obs['description'] = od_obs['o_city'].astype('str')+' to '+od_obs['d_city'].astype('str')
df = pd.merge(df, od_obs[['otaz','dtaz','description']], on=['otaz','dtaz'])
df.index = df.description
df= df.drop('description', axis=1)
df = df[['travel_time_all','travel_time_rail']]
df.rename(columns={'travel_time_all': 'All Transit Travel Time', 'travel_time_rail': 'Rail Path Travel Time'}, inplace=True)
df.index.name = None
df

,All Transit Travel Time,Rail Path Travel Time
Seattle to Bellevue,45.5,41.9
Seattle to Bellevue,45.5,41.9
Seattle to Everett,73.6,66.2
Seattle to Everett,73.6,66.2
Everett to Seattle,92.6,85.9
Everett to Seattle,92.6,85.9
Everett to Bellevue,121.8,82.7
Everett to Bellevue,121.8,82.7
Bellevue to Seattle,47.5,45.3
Bellevue to Seattle,47.5,45.3


**PM Peak (5 to 6)**

In [10]:
skim_name = '17to18'

df_all = results_dict[skim_name]['r']
df_all['mode'] = 'r'
df_rail = results_dict[skim_name]['a']
df_rail['mode'] = 'a'
df = pd.merge(df_all,df_rail,on=['otaz','dtaz'], suffixes=['_all','_rail'])
df
# Take minimum travel time 
df['travel_time_min'] = df[['travel_time_all','travel_time_rail']].min(axis=1)

# # join observed data to get 
od_obs['description'] = od_obs['o_city'].astype('str')+' to '+od_obs['d_city'].astype('str')
df = pd.merge(df, od_obs[['otaz','dtaz','description']], on=['otaz','dtaz'])
df.index = df.description
df= df.drop('description', axis=1)
df = df[['travel_time_all','travel_time_rail']]
df.rename(columns={'travel_time_all': 'All Transit Travel Time', 'travel_time_rail': 'Rail Path Travel Time'}, inplace=True)
df.index.name = None
df

,All Transit Travel Time,Rail Path Travel Time
Seattle to Bellevue,50.2,46.5
Seattle to Bellevue,50.2,46.5
Seattle to Everett,90.7,83.3
Seattle to Everett,90.7,83.3
Everett to Seattle,75.7,69.1
Everett to Seattle,75.7,69.1
Everett to Bellevue,105.8,83.7
Everett to Bellevue,105.8,83.7
Bellevue to Seattle,49.5,45.5
Bellevue to Seattle,49.5,45.5
